In [5]:
# rating 데이터로 행렬 인수분해를 통해서 누락값들을 예측하고 영화 추천
import pandas as pd
import numpy as np



In [4]:
ratings = pd.read_csv('data/ratings.csv')
df = ratings.pivot_table(index = 'userId', columns = 'movieId', values = 'rating', aggfunc = 'sum').values
df

array([[4. , nan, 4. , ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [2.5, 2. , 2. , ..., nan, nan, nan],
       [3. , nan, nan, ..., nan, nan, nan],
       [5. , nan, nan, ..., nan, nan, nan]])

In [6]:
mask = ~np.isnan(df)
mask

array([[ True, False,  True, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [ True,  True,  True, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False]])

In [10]:
k = 2
m, n = df.shape

# 아무 값을 줘놓고 경사하강법을 통해 수렴해 나갈 것임.
U = np.random.rand(m, k)
V = np.random.rand(n, k)  # k행 n열이 맞음. 계산을 위해 이렇게 표현

U @ V.T

array([[0.67783019, 0.63485367, 1.25482143, ..., 0.89273765, 0.27623074,
        1.31265324],
       [0.59891816, 0.61642452, 1.23340343, ..., 0.84572395, 0.24280283,
        1.34993018],
       [0.46011919, 0.53119467, 1.07705293, ..., 0.70884767, 0.18521477,
        1.23453381],
       ...,
       [0.10126393, 0.23242051, 0.49660822, ..., 0.27451268, 0.03811917,
        0.66749385],
       [0.1677825 , 0.27700291, 0.57993446, ..., 0.34394318, 0.0656324 ,
        0.73559957],
       [0.38849671, 0.30937105, 0.59674654, ..., 0.45576422, 0.15956792,
        0.56562768]])

In [12]:
lr = 0.00001  # 학습률

for i in range(5000):
    df_pred = U @ V.T

    # 오차마스크
    E = np.zeros((m, n))
    E[mask] = df_pred[mask] - df[mask]  # True인 부분만 접근한다.

    U_grad = E @ V
    V_grad = E.T @ U

    U -= U_grad * lr
    V -= V_grad * lr

    loss = sum(E[mask] ** 2)
    print(loss)

1037897.849256881
1032496.3905492953
1027139.9931584344
1021827.5737938149
1016558.0859613182
1011330.5187841835
1006143.8958685115
1000997.274211092
995889.7431475534
990820.4233388444
985788.4657943082
980793.0509295955
975833.3876576996
970908.7125117829
966018.2887980313
961161.4057774764
956337.3778750689
951545.5439153315
946785.266382647
942055.9307059372
937356.9445657991
932687.7372238411
928047.7588726553
923436.4800060939
918853.390808411
914298.0005619356
909769.8370721306
905268.4461097069
900793.3908686093
896344.2514396803
891920.6242990224
887522.1218106677
883148.3717428872
878799.0167976922
874473.7141529168
870172.135016499
865893.9641923564
861638.8996575982
857406.6521506816
853196.9447696257
849009.5125807533
844844.1022370511
840700.4716055073
836578.3894041758
832477.6348474906
828397.9973003032
824339.2759400717
820301.2794269389
816283.8255817144
812286.7410710666
808309.8611001997
804353.0291124672
800416.0964958785
796498.9222962207
792601.3729367654
788723.

In [13]:
# nan값을 경사하강법을 통한 예측값을 통해 채우기
df_filled = df.copy()

df_filled[~mask] = df_pred[~mask]
df_filled

array([[4.        , 4.26673543, 4.        , ..., 3.18615138, 1.96146773,
        4.39769891],
       [3.75902226, 3.3244488 , 3.17122608, ..., 2.45841189, 1.51989431,
        3.44789551],
       [2.36846382, 2.07118675, 1.98512731, ..., 1.4742356 , 0.92692536,
        2.19908763],
       ...,
       [2.5       , 2.        , 2.        , ..., 2.33277052, 1.45629782,
        3.39120341],
       [3.        , 2.95940702, 2.83163376, ..., 2.13580771, 1.3346585 ,
        3.1160808 ],
       [5.        , 3.92290792, 3.73915261, ..., 2.91897889, 1.7997757 ,
        4.05257703]])

In [14]:
# 컴퓨터가 예측해준 값들만 넣어 둠.
zeros = np.zeros(df_pred.shape)
zeros[~mask] += df_pred[~mask]
zeros

array([[0.        , 4.26673543, 0.        , ..., 3.18615138, 1.96146773,
        4.39769891],
       [3.75902226, 3.3244488 , 3.17122608, ..., 2.45841189, 1.51989431,
        3.44789551],
       [2.36846382, 2.07118675, 1.98512731, ..., 1.4742356 , 0.92692536,
        2.19908763],
       ...,
       [0.        , 0.        , 0.        , ..., 2.33277052, 1.45629782,
        3.39120341],
       [0.        , 2.95940702, 2.83163376, ..., 2.13580771, 1.3346585 ,
        3.1160808 ],
       [0.        , 3.92290792, 3.73915261, ..., 2.91897889, 1.7997757 ,
        4.05257703]])

In [16]:
movies = pd.read_csv('data/movies.csv')
movie_names = {}

for i in range(len(movies)):
    a = movies.iloc[i]
    movie_names[a['movieId']] = a['title']

In [18]:
for i in range(len(zeros)):
    a = zeros[i]
    for j in range(len(a)):
        if a[j] >= 4:
            if j in movie_names.keys():
                print(f"{i}번째 유저, {movie_names[j]} 추천!")
            else:
                print(f"{i}번째 유저, {j}번째 영화 추천!")

0번째 유저, Toy Story (1995) 추천!
0번째 유저, Sudden Death (1995) 추천!
0번째 유저, GoldenEye (1995) 추천!
0번째 유저, Balto (1995) 추천!
0번째 유저, Cutthroat Island (1995) 추천!
0번째 유저, Casino (1995) 추천!
0번째 유저, Sense and Sensibility (1995) 추천!
0번째 유저, Money Train (1995) 추천!
0번째 유저, Get Shorty (1995) 추천!
0번째 유저, Powder (1995) 추천!
0번째 유저, Leaving Las Vegas (1995) 추천!
0번째 유저, Othello (1995) 추천!
0번째 유저, Now and Then (1995) 추천!
0번째 유저, Persuasion (1995) 추천!
0번째 유저, Shanghai Triad (Yao a yao yao dao waipo qiao) (1995) 추천!
0번째 유저, Dangerous Minds (1995) 추천!
0번째 유저, Twelve Monkeys (a.k.a. 12 Monkeys) (1995) 추천!
0번째 유저, 33번째 영화 추천!
0번째 유저, 35번째 영화 추천!
0번째 유저, Dead Man Walking (1995) 추천!
0번째 유저, 37번째 영화 추천!
0번째 유저, Richard III (1995) 추천!
0번째 유저, Seven (a.k.a. Se7en) (1995) 추천!
0번째 유저, Pocahontas (1995) 추천!
0번째 유저, Usual Suspects, The (1995) 추천!
0번째 유저, Mighty Aphrodite (1995) 추천!
0번째 유저, Big Green, The (1995) 추천!
0번째 유저, Georgia (1995) 추천!
0번째 유저, Eye for an Eye (1996) 추천!
0번째 유저, Bio-Dome (1996) 추천!
0번째 유저, Lawnmower Ma

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)




596번째 유저, 2999번째 영화 추천!
596번째 유저, 3001번째 영화 추천!
596번째 유저, My Best Fiend (Mein liebster Feind) (1999) 추천!
596번째 유저, Insider, The (1999) 추천!
596번째 유저, American Movie (1999) 추천!
596번째 유저, They Shoot Horses, Don't They? (1969) 추천!
596번째 유저, 3012번째 영화 추천!
596번째 유저, Coma (1978) 추천!
596번째 유저, Re-Animator (1985) 추천!
596번째 유저, General, The (1926) 추천!
596번째 유저, Piranha (1978) 추천!
596번째 유저, Trading Places (1983) 추천!
596번째 유저, 3050번째 영화 추천!
596번째 유저, Felicia's Journey (1999) 추천!
596번째 유저, Where's Marlowe? (1998) 추천!
596번째 유저, Poison Ivy (1992) 추천!
596번째 유저, Stand and Deliver (1988) 추천!
596번째 유저, All About My Mother (Todo sobre mi madre) (1999) 추천!
596번째 유저, My Man Godfrey (1957) 추천!
596번째 유저, 3123번째 영화 추천!
596번째 유저, 3124번째 영화 추천!
596번째 유저, 3128번째 영화 추천!
596번째 유저, 3131번째 영화 추천!
596번째 유저, Great Santini, The (1979) 추천!
596번째 유저, 3136번째 영화 추천!
596번째 유저, Hell in the Pacific (1968) 추천!
596번째 유저, Green Mile, The (1999) 추천!
596번째 유저, Last Picture Show, The (1971) 추천!
596번째 유저, Simpatico (1999) 추천!
596번째 

In [21]:
df2 = ratings.pivot_table(index = 'userId', columns = 'movieId', values = 'rating', aggfunc = 'sum')
df2

columns = df2.columns

user = 0
a = zeros[user]
for i in range(len(a)):
    if a[i] > 4.8:
        idx = columns[i]
        name = movie_names[idx]
        print(f'{name} 추천')

Casino (1995) 추천
Sense and Sensibility (1995) 추천
Persuasion (1995) 추천
City of Lost Children, The (Cité des enfants perdus, La) (1995) 추천
Twelve Monkeys (a.k.a. 12 Monkeys) (1995) 추천
Cry, the Beloved Country (1995) 추천
Postman, The (Postino, Il) (1994) 추천
Angels and Insects (1995) 추천
White Squall (1996) 추천
Beautiful Girls (1996) 추천
Hate (Haine, La) (1995) 추천
Taxi Driver (1976) 추천
Crumb (1994) 추천
Kids (1995) 추천
Living in Oblivion (1995) 추천
Party Girl (1995) 추천
Before Sunrise (1995) 추천
Hoop Dreams (1994) 추천
Heavenly Creatures (1994) 추천
Like Water for Chocolate (Como agua para chocolate) (1992) 추천
Madness of King George, The (1994) 추천
Murder in the First (1995) 추천
New York Cop (Nyû Yôku no koppu) (1993) 추천
Once Were Warriors (1994) 추천
Léon: The Professional (a.k.a. The Professional) (Léon) (1994) 추천
Three Colors: Red (Trois couleurs: Rouge) (1994) 추천
Three Colors: White (Trzy kolory: Bialy) (1994) 추천
Shawshank Redemption, The (1994) 추천
Vanya on 42nd Street (1994) 추천
Lion King, The (1994) 추천